In [ ]:
!pip install anvil-uplink

In [1]:
from keras.models import load_model
import unidecode
import contractions
import re
from word2number import w2n
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

2023-01-16 14:54:34.847125: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chriskinyash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/chriskinyash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/chriskinyash/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from transformers import TFBertModel,BertConfig, BertTokenizerFast
from transformers import AlbertConfig, AlbertTokenizerFast, TFAlbertModel
import os
import numpy as np 
import uuid
import anvil.server

/home/chriskinyash/anaconda3/envs/TF/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class TFBertMainLayer(tf.keras.layers.Layer):
    def __init__(self, config, **kwargs):
        super(TFBertMainLayer, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained('bert-base-uncased', config=config)
        
    def call(self, inputs):
        return self.bert(inputs)

In [4]:
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
    text = input_text,
    add_special_tokens=True,
    max_length=100,
    truncation=True,
    padding='max_length',
    return_tensors='tf',
    verbose = True)
    return {
     'input_ids':tf.cast(token.input_ids, tf.float64),
     'attention_mask': tf.cast(token.attention_mask, tf.float64)
  }

In [5]:
def preprocess_text(text):
    """ A function to preprocess text"""
    #convert Non-ASCII characters to ASCII
    text = unidecode.unidecode(text)
    
    # Expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)

    # removes special characters from text, e.g. $"""
    clean = re.compile(r'[^a-zA-Z0-9\s]')
    text = re.sub(clean, ' ', text)
    # converts characters to lowercase"""
    text = text.lower()
    
    # remove patterns
    text = re.sub(r'xxxx', r'', text)
    
    # Convert number words to digits and remove them"""
    
    pattern = r'(\W+)'
    tokens = re.split(pattern, text)

    for i, token in enumerate(tokens):
        try:
            tokens[i] = str(w2n.word_to_num(token))
        except:
            pass
    
    text = ''.join(tokens)

    # removes numbers from text"""
    clean_1 = re.compile(r'\d+')
    
    text = re.sub(clean_1, '', text)

    # removes words with length 1 or 2"""
    clean = re.compile(r'\b\w{1,2}\b')
    
    text = re.sub(clean, '', text)
    
    # removes the stopwords in the text"""
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords = set(stopwords)
    
    tokens = re.split(r'(\W+)', text)
    
    text = [token for token in tokens if token not in stopwords]

    text = ' '.join(text)

    # Remove extra spaces from a string"""
    
    clean = re.compile(r'\s{2,10000}')
    text = re.sub(clean, ' ', text)


    lemma = WordNetLemmatizer()
    
    tokens = re.split('\W+', text)
    
    text = [lemma.lemmatize(token) for token in tokens]
    
    text = ' '.join(text)
    
    return text

In [6]:
# We define out classes for ease of modelling 
class Warehouse_Model(object):
    """Class to save your model architecture and Model Weights correctly"""
    def __init__(self):
        pass
   
    def save(model,file_path):
        """
        Function for saving both the model and weights to your file_path
        Follow this structure ['/file_path/name_you_want_to_save_as']
        """
        SaveModel = tf.saved_model.save(model, file_path)
        
        
        SaveWeights = model.save_weights(file_path + '.h5')
        
        return print(f'Model and Weights saved SUCCESSFULLY in:{file_path}')
    
    def load(file_path):
        
        """Follow this structure ['/file_path/name_of_file containing your model!']"""
        # Load The Model
        LoadModel = tf.keras.models.load_model("model_path")
        
        return LoadModel

In [7]:
import tensorflow as tf
Best_Model = tf.keras.models.load_model('ALBERT_SGD')

2023-01-16 14:54:37.743206: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-16 14:54:37.743320: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-16 14:54:37.770172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-16 14:54:37.770318: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:09:00.0 name: NVIDIA GeForce GTX TITAN X computeCapability: 5.2
coreClock: 1.2155GHz coreCount: 24 deviceMemorySize: 11.92GiB deviceMemoryBandwidth: 313.37GiB/s
2023-01-16 14:54:37.770328: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-01-16 14:54:37.771067: I tensorflow/stream_executor/platfo

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpfwgpr0eb.py, line 115)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpfwgpr0eb.py, line 115)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [8]:
# Defining a Class for loading transformers from Hugging face.
class LoadBert(object):
    
    """A class that loads a bert model, tokenizer and a transformer"""
    def __init__(self):
        pass
    def loadModel(modelName):
        """Loading the model name"""
        model = modelName
        return model

    def config (Bert_Configuration, modelName, outputhidden_states:bool):
        """Model configuration"""
        config = Bert_Configuration.from_pretrained(modelName)
        config.outputhidden_states = False
        return config

    def tokenizer(tokenizer, modelName, config):
        """Text tokenizer"""
        tokenizer = tokenizer.from_pretrained(pretrained_model_name_or_path = modelName, config =config)
        return tokenizer

    def transformerModel(BertClass, modelName, config):
        """Hugging face transformer model"""
        transformerModel = BertClass.from_pretrained(modelName, config)
        return transformerModel

In [9]:
# Loading ALbert
ALBERT_v2 = LoadBert

# Tokenizer
tokenize_BERT = ALBERT_v2.tokenizer(tokenizer=AlbertTokenizerFast,
                                    config=AlbertConfig,
                                    modelName='albert-base-v2')

# Transformer Model
albert_model = ALBERT_v2.transformerModel(BertClass=TFAlbertModel, 
                                          config=AlbertConfig,
                                          modelName='albert-base-v2')

# Model configuration
config_ALBERT = ALBERT_v2.config(AlbertConfig, 
                                 modelName='albert-base-v2', 
                                 outputhidden_states=False)

2023-01-16 14:54:46.247527: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
Some layers from the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [10]:
timely_response_dict ={
 0: 'No',
 1: 'Yes'}

product_dict ={
 0: 'Bank account or service',
 1: 'Consumer Loan',
 2: 'Credit card',
 3: 'Credit reporting',
 4: 'Debt collection',
 5: 'Money transfers',
 6: 'Mortgage',
 7: 'Other financial service',
 8: 'Payday loan',
 9: 'Prepaid card',
 10: 'Student loan',
 11: 'Virtual currency'}

disputed_dict ={
 0: 'No', 1: 'Yes'}

company_response_dict = {
 0: 'Closed',
 1: 'Closed with explanation',
 2: 'Closed with monetary relief',
 3: 'Closed with non-monetary relief',
 4: 'Untimely response'}

In [11]:
anvil.server.connect("YQHV3BXLTQKWBYWUAA62TNML-RH7IVEVSJV6MZMCA")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:

@anvil.server.callable


def predict(input_text):

    # Preprocess the text
    clean_text = preprocess_text(input_text)
    # Tokenize the input text
    tokenized_text_input = prepare_data(input_text=clean_text, tokenizer=tokenize_BERT)
    
    # Create the prediction dictionary to return
    prediction = {
        "timely_response": None,
        "product": None,
        "company_response": None,
        "disputed": None
    }
       # Use the model to make a prediction
    preds = Best_Model.predict(tokenized_text_input)
    

    timely_response_prediction = np.argmax(preds['timely_response'])
    product_prediction = np.argmax(preds['product'])
    company_response_prediction = np.argmax(preds['company_response'])
    disputed_prediction = np.argmax(preds['disputed'])
    
    
    
    prediction["timely_response"] = timely_response_dict[int(timely_response_prediction)]
    prediction["product"] = product_dict[int(product_prediction)]
    prediction["company_response"] = company_response_dict[int(company_response_prediction)]
    prediction["disputed"] = disputed_dict[int(disputed_prediction)]

# Return the prediction as a JSON response
    return prediction

anvil.server.wait_forever()   

2023-01-16 14:55:27.984870: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-01-16 14:55:28.004892: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3693190000 Hz
/home/chriskinyash/anaconda3/envs/TF/lib/python3.9/site-packages/tensorflow/python/keras/engine/functional.py:592: UserWarning: Input dict contained keys ['attention_mask'] which did not match any model input. They will be ignored by the model.
  warnings.warn(
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/chriskinyash/anaconda3/envs/TF/lib/python3.9/site-packages/anvil/server.py", line 405, in call
    return _do_call(args, kwargs, fn_name=fn_name)
  File "/home/chriskinyash/anaconda3/envs/TF/lib/python3.9/site-packages/anvil/server.py", line 395, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
  File "/home/chriskinyash/anaconda3

Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [13]:

text = ' made balance transfer onto chase credit card interest rate total subsequently making minimum payment well additional payment cover purchase recently found payment applied pursuant credit card bill right minimum payment applied lower rate payment minimum also applied lower rate instead higher rate required balance instead balance higher rate payment misapplied '
# Preprocess the text
clean_text = preprocess_text(text)



# Tokenize the input text
tokenized_text_input = prepare_data(input_text=clean_text, tokenizer=tokenizer)
preds = Best_Model.predict(tokenized_text_input)

In [14]:
preds

{'product': array([[0.5540818 , 0.47563013, 0.60292786, 0.81291145, 0.854082  ,
         0.34129182, 0.6212632 , 0.27046806, 0.3122816 , 0.29392916,
         0.47677657, 0.27023846]], dtype=float32),
 'company_response': array([[0.25550836, 0.8570209 , 0.3731316 , 0.59634316, 0.28639954]],
       dtype=float32),
 'disputed': array([[0.5518131 , 0.22720681]], dtype=float32),
 'timely_response': array([[0.1686151, 0.7738379]], dtype=float32)}

In [19]:

prediction = {
        "timely_response": None,
        "product": None,
        "company_response": None,
        "disputed": None
    }
timely_response_prediction = np.argmax(preds['timely_response'])
product_prediction = np.argmax(preds['product'])
company_response_prediction = np.argmax(preds['company_response'])
disputed_prediction = np.argmax(preds['disputed'])
    
    
    
prediction["timely_response"] = timely_response_dict[int(timely_response_prediction)]
prediction["product"] = product_dict[int(product_prediction)]
prediction["company_response"] = company_response_dict[int(company_response_prediction)]
prediction["disputed"] = disputed_dict[int(disputed_prediction)]


In [20]:
prediction

{'timely_response': 'Yes',
 'product': 'Debt collection',
 'company_response': 'Closed with explanation',
 'disputed': 'No'}